# 01 • Auditoria da Base de Imóveis

**Objetivo:** tamanho, colunas, nulos e duplicidades (um imóvel pode ter múltiplos titulares).

Checklist:
- ler amostra
- % nulos
- duplicidade por `CÓDIGO DO IMOVEL`
- contagem total via chunks


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re, unicodedata

# Caminhos do projeto (ajuste se necessário)
BASE_DIR = Path.cwd().resolve().parents[0]  # .../notebooks -> projeto
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"
OUTPUTS_DIR = BASE_DIR / "outputs"

RAW_DIR, PROCESSED_DIR, OUTPUTS_DIR


(WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/raw'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/processed'),
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/outputs'))

In [2]:
imoveis_csv = RAW_DIR / "Imoveis_SP_01_02_2026.csv"
imoveis_csv.exists(), imoveis_csv

(True,
 WindowsPath('C:/Users/User/Desktop/python/ali_rural_prospecção/data/raw/Imoveis_SP_01_02_2026.csv'))

In [3]:
df = pd.read_csv(imoveis_csv, sep=";", nrows=2000, dtype=str, low_memory=False)
df.head(3)

,CÓDIGO DO IMOVEL,DENOMIÇÃO DO IMÓVEL,CÓDIGO DO MUNICÍPIO (IBGE),MUNICÍPIO,UF,ÁREA TOTAL,TITULAR,NATUREZA JURÍDICA,CONDIÇÃO DA PESSOA,PERCENTUAL DE DETENÇÃO,PAÍS
0,191949803,FAZENDA SAO SEBASTIAO - GLEBA II,3517703,GUARÁ,SP,"58,4024",ROSA ********************,NaN,Proprietario Ou Posseiro Comum,20,BRASIL
1,191949803,FAZENDA SAO SEBASTIAO - GLEBA II,3517703,GUARÁ,SP,"58,4024",SEBASTIAO ********************,NaN,Proprietario Ou Posseiro Comum,20,BRASIL
2,191949803,FAZENDA SAO SEBASTIAO - GLEBA II,3517703,GUARÁ,SP,"58,4024",PAULO ********************,NaN,Proprietario Ou Posseiro Comum,20,BRASIL


In [4]:
df.columns.tolist()

['CÓDIGO DO IMOVEL',
 'DENOMIÇÃO DO IMÓVEL',
 'CÓDIGO DO MUNICÍPIO (IBGE)',
 'MUNICÍPIO',
 'UF',
 'ÁREA TOTAL',
 'TITULAR',
 'NATUREZA JURÍDICA',
 'CONDIÇÃO DA PESSOA',
 'PERCENTUAL DE DETENÇÃO',
 'PAÍS']

In [5]:
(df.isna().mean()*100).sort_values(ascending=False)

NATUREZA JURÍDICA             96.50
PAÍS                          58.25
PERCENTUAL DE DETENÇÃO         1.80
CÓDIGO DO IMOVEL               0.00
DENOMIÇÃO DO IMÓVEL            0.00
CÓDIGO DO MUNICÍPIO (IBGE)     0.00
MUNICÍPIO                      0.00
UF                             0.00
ÁREA TOTAL                     0.00
TITULAR                        0.00
CONDIÇÃO DA PESSOA             0.00
dtype: float64

In [6]:
df["CÓDIGO DO IMOVEL"].value_counts().head(10)

CÓDIGO DO IMOVEL
192793742    14
192691400    13
192699304    12
192648904     9
192743214     9
192774527     9
192740207     8
192752710     8
192649200     8
192679885     7
Name: count, dtype: int64

## Contagem total (chunks)

In [7]:
total = 0
for chunk in pd.read_csv(imoveis_csv, sep=";", dtype=str, chunksize=200_000, low_memory=False):
    total += len(chunk)
total

870011